# Module 5 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

## Solving Normal Form Games

In [1]:
from typing import List, Tuple, Dict, Callable
from copy import deepcopy

In the lecture we talked about the Prisoner's Dilemma game, shown here in Normal Form:

Player 1 / Player 2  | Defect | Cooperate
------------- | ------------- | -------------
Defect  | -5, -5 | -1, -10
Cooperate  | -10, -1 | -2, -2

where the payoff to Player 1 is the left number and the payoff to Player 2 is the right number. We can represent each payoff cell as a Tuple: `(-5, -5)`, for example. We can represent each row as a List of Tuples: `[(-5, -5), (-1, -10)]` would be the first row and the entire table as a List of Lists:

In [2]:
prisoners_dilemma = [
 [( -5, -5), (-1,-10)],
 [(-10, -1), (-2, -2)]]

prisoners_dilemma

[[(-5, -5), (-1, -10)], [(-10, -1), (-2, -2)]]

in which case the strategies are represented by indices into the List of Lists. For example, `(Defect, Cooperate)` for the above game becomes `prisoners_dilemma[ 0][ 1]` and returns the payoff `(-1, -10)` because 0 is the first row of the table ("Defect" for Player 1) and 1 is the 2nd column of the row ("Cooperate" for Player 2).

For this assignment, you are going write a function that uses Successive Elimination of Dominated Strategies (SEDS) to find the **pure strategy** Nash Equilibrium of a Normal Form Game. The function is called `solve_game`:

```python
def solve_game( game: List[List[Tuple]], weak=False) -> List[Tuple]:
    pass # returns strategy indices of Nash equilibrium or None.
```

and it takes two parameters: the game, in a format that we described earlier and an optional boolean flag that controls whether the algorithm considers only **strongly dominated strategies** (the default will be false) or whether it should consider **weakly dominated strategies** as well.

It should work with game matrices of any size and it will return the **strategy indices** of the Nash Equilibrium. If there is no **pure strategy** equilibrium that can be found using SEDS, return the empty List (`[]`).


<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
Do not return the payoff. That's not useful. Return the strategy indices, any other output is incorrect.
</div>

As before, you must provide your implementation in the space below, one Markdown cell for documentation and one Code cell for implementation, one function and assertations per Codecell.


---

<a id="find_valid_successors"></a>
## find_valid_successors
`find_valid_successors` takes the game and a tuple of a list of current leftover rows, and a list of current leftover columns. Also takes a boolean indicating whether we consider weakly dominated strategies. Returns a list of valid successors from the current game state.
We can either elimate one row or one column.
We check each row whether it's dominated by another row.
And we do the same for the columns.
**Used by**: [solve_game](#solve_game)

* **game** List[List[Tuple]] a 2d list of Tuples, each tuple represent the score for player 1, player 2 at the corresponding strategies.
* **current_game_state**: a tuple containing the current leftover rows (a list of ints) and columns (a list of ints) 
* **weak**: a boolean showing whether we consider weakly dominated strategies.

**returns** a list of valid successor game states.

Psudocode is as follows for the whole function:
1. For each row, we check whether it's dominated by another row, if weak is true we use <= and if it's false we use ==.
   Note that if all of the elements in a row are equal to another row, they are still both weakly-dominated in this consideration
   If it's the only row, we ignore it.
3. If it is, we add it (game, list of rows minus the current row, list of columns),
4. We do the same for each column

In [3]:
def find_valid_successors(game: List[List[Tuple]], current_game_state:Tuple[List[int],List[int]], weak:bool=False) ->List[Tuple[List[int],[List[int]]]]:
    valid_successors = []
    for row in current_game_state[0]:
        for other_row in current_game_state[0]:
            if row != other_row:
                if all((weak and game[row][c][0] <= game[other_row][c][0]) or (not weak and game[row][c][0] < game[other_row][c][0]) 
                for c in current_game_state[1]):
                    valid_successors.append(( [r for r in current_game_state[0] if r != row], current_game_state[1]))
    for col in current_game_state[1]:
        for other_col in current_game_state[1]:         
            if col != other_col:
                if all((weak and game[r][col][1] <= game[r][other_col][1]) or (not weak and game[r][col][1] < game[r][other_col][1])
                for r in current_game_state[0]):
                    valid_successors.append((current_game_state[0],  [c for c in current_game_state[1] if c != col]))
    return valid_successors

In [4]:
game = prisoners_dilemma
assert((find_valid_successors(game, ([0,1],[0,1]), False)) == [([0], [0, 1]), ([0, 1], [0])])
game = [
[(1, 0), (3, 1)],
[(1, 1), (3, 0)]]
assert((find_valid_successors(game, ([0,1],[0,1]),True)) == [([1], [0, 1]), ([0], [0, 1])])
game = [
[(2, 1), (3, 1)],
[(2, 1), (3, 0)]]
assert((find_valid_successors(game, ([1],[0,1]),True)) == [([1], [0])])

<a id="solve_game"></a>
## solve_game
`solve_game` takes a 2d list of tuples and an optional boolean value, output a list of all Nash Equilibrium positions given whether we want to consider weakly dominated strategies. We use State Space Search to solve this problem. Essentially DFS. 
 **Used by**: test games 1-4

* **game**: a 2d list of Tuples, each tuple represent the score for player 1, player 2 at the corresponding strategies.
* **weak**: whether we consider weakly dominated strategies.

**returns** list of tuples each tuple represent a posiition in the original game that is a pure strategy Nash Equilibrium.

Psudocode is as follows for the whole function:
1. Initialize the starting state: set frontier to be a stack of the following structure: A tuple game_state where game_state[0]= a list of ints for the current available rows, and game_state[1] = a list of ints for the current available columns.
2. We put the starting game_state on the frontier
3. While the frontier is not empty we do the following:
   * We pop the last element from the frontier: current_game_state
   * We find valid successors of current_game_state (eliminating one row or one column that is dominated)
   * We check if the successor is only one element, if so, we put it on the solution
   * Otherwise we append it to the frontier.
4. We return the final solution array

In [5]:
def solve_game(game: List[List[Tuple]], weak:bool=False) -> List[Tuple]:
    frontier = [(list(range(0, len(game))), list(range(0, len(game[0]))))]
    solutions = set()
    while len(frontier) > 0:
        current_game_state = frontier.pop()
        successors = find_valid_successors(game, current_game_state, weak)
        for successor in successors:
            if len(successor[0])==1 and len(successor[1])==1 :
                solutions.add((successor[0][0],successor[1][0]))
            else:
                frontier.append(successor)
    return list(solutions)

## Additional Directions

Create three games as described and according to the following:

1. Your games must be created and solved "by hand".
2. The strategy pairs must **not** be on the main diagonal (0, 0), (1, 1), or (2, 2). And the solution cannot be the same for both Game 1 and Game 2.
3. Make sure you fill out the Markdown ("?") with your game as well as the solution ("?").
4. Remember, **do not return the payoff**, return the strategy indices (a list of them).

## Before you code...

Solve the following game by hand using SEDS and weakly dominated strategies. 
The game has three (pure) Nash Equilibriums. 
You should find all of them.
This will help you think about what you need to implement to make the algorithm work.
**Hint**: You will need State Space Search from Module 1 and SEDS from Module 5 to get the full algorithm to work.

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|1  | 1/1 | 3/0 | 0/1 |
|2  | 2/2 | 3/3 | 0/2 |

**Solutions**: 
Let's start with player 1, we can see that strategy 1 is weakly dominated by strategy 0 (also strategy 2 but it doesn't matter), with 
(1,0) = 1 <= (0,0) = 1; (1,1) = 3 <= (0,1) = 3, and (1,2) = 0 <= (0,2) = 1
So we eliminate startegy 1 for player 1 and is left with the following table: 

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|2  | 2/2 | 3/3 | 0/2 |

Now let's switch to player 2, we see that strategy 0 is strongly dominated by strategy 1 (also strategy 2 is weakly dominatated by strategy 1 but we go in order, this is essentially a form of DFS, so we will go back to this), with (0,0) = 0 < (0,1) = 1; (2,0) = 2 < (2,1) = 3. So we eliminate strategy 0 and get the following table:
| Player 1 / Player 2  | 1 | 2 |
|----|----|----|
|0  | 3/1 | 1/1 |
|2  | 3/3 | 0/2 |

We can either continue with player 2 or we can switch to player 1, so let's do player 1 first: we see that strategy 2 is weakly dominated by strategy 0, with (2,1) = 3 <= (0,1) = 3; (2,2) = 0 <= (0,2) = 1. So we eliminate strategy 2, and we get the following: 

| Player 1 / Player 2  | 1 | 2 |
|----|----|----|
|0  | 3/1 | 1/1 |

So now player 2 can pick either strategy as they are equal. So both of these are Nash equilibriums. We insert them to solutions.

If we go back and pick player 2 to continue, we can eliminate stragety 2 for player 2 as it's weakly dominated by strategy 1: (0,2) = 1 <= (0,1) = 1; (2,2) = 2 <= (2,1) = 3. So we get: 

| Player 1 / Player 2  | 1 |
|----|----|
|0  | 3/1 |
|2  | 3/3 |

We see that player 1 can pick either strategy and earn the same score of 3. So they are both Nash equilibriums. Therefore we insert them into the solution and we get the solution is {(0,1), (0,2), (2,1)}, notice there could be duplicates in the process so we use a set to track the solutions and convert to a list at the end.

We technically haven't finished yet, but if we go back to the original table, we see that if we pick player 2 first, there is no strategy that's dominated by another strategy. So this is the solution: [(0,1), (0,2), (2,1)]

### Test Game 1. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Strongly Dominated Strategies**
With the current implementation, it's impossible for a problem to be only solvable using Successive Elimintation of Strongly Dominated Strategies. 
Weakly dominated strategies strickly explores broader states.

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 3/2 | 2/3 | 0/4 |
|1  | 4/1 | 3/2 | 1/3 |
|2  | 1/1 | 4/4 | 2/3 |

**Solution:** [2,1]

In [6]:
test_game_1 = [
[(3, 2), (2, 3), (0, 4)],
[(4, 1), (3, 2), (1, 3)],
[(1, 1), (4, 4), (2, 3)]]

solution = solve_game(test_game_1)

In [7]:
assert solution == [(2,1)]

### Test Game 2. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Weakly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/1 | 2/2 | 3/3 |
|1  | 2/2 | 3/3 | 4/4 |
|2  | 3/3 | 5/5 | 4/4 |

**Solution:** [(2,1)]

In [8]:
test_game_2 =  [
[(1, 1), (2, 2), (3, 3)],
[(2, 2), (3, 3), (4, 4)],
[(3, 3), (5, 5), (4, 4)]]

strong_solution = solve_game( test_game_2)
weak_solution = solve_game(test_game_2, weak=True)

In [9]:
assert strong_solution == []
assert weak_solution == [(2, 1)]

### Test Game 3. Create a 3x3 two player game

**that cannot be solved using the Successive Elimintation of Dominated Strategies at all**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|1  | 1/1 | 2/0 | 2/1 |
|2  | 2/2 | 3/3 | 0/0 |

**Solution:** None

In [10]:
test_game_3 = [
[(1, 0), (3, 1), (1, 1)],
[(1, 1), (2, 0), (2, 1)],
[(2, 2), (3, 3), (0, 0)]]


strong_solution = solve_game( test_game_3)
weak_solution = solve_game( test_game_3, weak=True)

In [11]:
assert strong_solution == []
assert weak_solution == []

### Test Game 4. Multiple Equilibria

You solve the following game by hand, above.
Now use your code to solve it.

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|1  | 1/1 | 3/0 | 0/1 |
|2  | 2/2 | 3/3 | 0/2 |

**Solutions:**  [(0,1), (0,2), (2,1)]

In [12]:
test_game_4 = [
[(1, 0), (3, 1), (1, 1)],
[(1, 1), (3, 0), (0, 3)],
[(2, 2), (3, 3), (0, 2)]]

strong_solution = solve_game( test_game_4)
weak_solution = solve_game( test_game_4, weak=True)

In [13]:
assert strong_solution == []
assert weak_solution == [(0,1), (0,2), (2,1)]

## Before You Submit...

1. Did you provide output exactly as requested? **Don't forget to fill out the Markdown tables with your games**.
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.